<a href="https://colab.research.google.com/github/Lishat/Rock_Paper_Scissors_Classification/blob/master/Rock_Paper_Scissors_Classification_in_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps.zip \
    -O /content/rps.zip
  
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps-test-set.zip \
    -O /content/rps-test-set.zip

--2019-07-01 05:36:59--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.141.128, 2607:f8b0:400c:c06::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.141.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 200682221 (191M) [application/zip]
Saving to: ‘/content/rps.zip’

/content/rps.zip    100%[===================>] 191.38M   185MB/s    in 1.0s    

2019-07-01 05:37:00 (185 MB/s) - ‘/content/rps.zip’ saved [200682221/200682221]

--2019-07-01 05:37:02--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps-test-set.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.141.128, 2607:f8b0:400c:c06::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.141.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29516758 (28M) [application/zip]
Saving to: ‘/content/rps-test-set.

In [0]:
import zipfile

local_zip = '/content/rps.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/')
zip_ref.close()

local_zip = '/content/rps-test-set.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/')
zip_ref.close()

In [0]:
import torch
from torchvision import transforms, datasets

In [0]:
data_transform = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
train_data = datasets.ImageFolder(root='/content/rps', transform = data_transform)
test_data = datasets.ImageFolder(root='/content/rps-test-set', transform = data_transform)
trainDataloader = torch.utils.data.DataLoader(train_data, batch_size=8,
                                              shuffle=True, num_workers=4)
testDataloader = torch.utils.data.DataLoader(test_data, batch_size=8,
                                             shuffle=True, num_workers=4)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [0]:
class Model(torch.nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self.conv = torch.nn.Sequential(torch.nn.Conv2d(3, 16, 5),
                                   torch.nn.ReLU(inplace=True),
                                   torch.nn.MaxPool2d(2),
                                   torch.nn.Conv2d(16, 32, 5),
                                   torch.nn.BatchNorm2d(32),
                                   torch.nn.ReLU(inplace=True),
                                   torch.nn.MaxPool2d(2),
                                   torch.nn.Conv2d(32, 64, 5),
                                   torch.nn.BatchNorm2d(64),
                                   torch.nn.ReLU(inplace=True),
                                   torch.nn.MaxPool2d(2),
                                   torch.nn.Conv2d(64, 128, 5),
                                   torch.nn.BatchNorm2d(128),
                                   torch.nn.ReLU(inplace=True),
                                   torch.nn.MaxPool2d(2),
                                   torch.nn.Conv2d(128, 256, 5),
                                   torch.nn.BatchNorm2d(256),
                                   torch.nn.ReLU(inplace=True),
                                   torch.nn.MaxPool2d(2))
    self.ann = torch.nn.Sequential(torch.nn.Linear(2304, 512),
                                   torch.nn.ReLU(inplace=True),
                                   torch.nn.Linear(512, 128),
                                   torch.nn.ReLU(inplace=True),
                                   torch.nn.Linear(128, 32),
                                   torch.nn.ReLU(inplace=True),
                                   torch.nn.Linear(32, 8),
                                   torch.nn.ReLU(inplace=True),
                                   torch.nn.Linear(8, 3),
                                   torch.nn.Softmax(),
                                  )
  def forward(self, x):
    x = self.conv(x)
    x = x.view(-1, 2304)
    x = self.ann(x)
    return x
    

In [0]:
myModel = Model()

In [17]:
myModel

Model(
  (conv): Sequential(
    (0): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
    (8): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
    (12): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inplace)
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (15): Conv2d(128, 256, kernel_

In [0]:
myModel.to(device)
Loss = torch.nn.CrossEntropyLoss()
Optimizer = torch.optim.Adam(myModel.parameters(), lr=0.00001)

In [22]:
for epoch in range(10):
  current_loss = 0.0
  for images, classes in trainDataloader:
    images = images.to(device)
    classes = classes.to(device)
    Optimizer.zero_grad()
    obtained_outputs = myModel(images)
    loss = Loss(obtained_outputs, classes)
    loss.backward()
    Optimizer.step()
    current_loss += loss.item()
  print("TRAINING:\nEPOCH:", epoch)
  print("LOSS:", current_loss)
with torch.no_grad():
  correctly_classified = 0
  total = 0
  for images, classes in testDataloader:
    images = images.to(device)
    classes = classes.to(device)
    correctly_classified += (torch.argmax(myModel(images), 1) == classes).sum().item()
    total += classes.size(0)
  print("OBTAINED ACCURACY:", correctly_classified*100/total)
    

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


TRAINING:
EPOCH: 0
LOSS: 214.35284048318863
TRAINING:
EPOCH: 1
LOSS: 213.2094103693962
TRAINING:
EPOCH: 2
LOSS: 215.49903893470764
TRAINING:
EPOCH: 3
LOSS: 215.68160820007324
TRAINING:
EPOCH: 4
LOSS: 212.68196028470993
TRAINING:
EPOCH: 5
LOSS: 211.7701252102852
TRAINING:
EPOCH: 6
LOSS: 213.71711760759354
TRAINING:
EPOCH: 7
LOSS: 211.60392796993256
TRAINING:
EPOCH: 8
LOSS: 214.26483511924744
TRAINING:
EPOCH: 9
LOSS: 212.49693179130554
OBTAINED ACCURACY: 81.45161290322581
